## Library
---

In [111]:
import os
import time
import numpy as np
from ase import Atoms, neighborlist, Atom
from ase.io import write

## Algorithm
---

In [2]:
'''
site energy -> activation energy -> diffusion rate

site energy(E_i) = -N*(E_b)/2

activation energy(E_a) = E_0 + alpha * E_r
E_r = E_i(end) - E_i(start)

diffusion rate = f*exp(-E_a/(k*T)) (Arrhenius)
f is ~ 10^13 for most metals
'''

def get_site_energy(bond_energy, bond_num):
    return -bond_num * bond_energy/2

def get_activation_energy(e_start, e_end, alpha = 0.1, e0 = 0):
    e_reaction = e_end-e_start
    if e_reaction>=0:
        return e0 + (1+alpha)*e_reaction
    else:
        return e0 + alpha*e_reaction

def get_diffusion_rate(e_a, T=300, f=1E13):
    k_B = 8.617333262145e-5  # Boltzmann constant in eV/K
    return f*np.exp(-e_a/(k_B*T))

In [13]:
# function to create fcc lattice
def create_fcc_lattice(symbol, lattice_constant, repetitions, vacuum=10):
    """
    Create an FCC lattice of atoms.

    Args:
    - symbol: The chemical symbol of the atoms.
    - lattice_constant: The lattice constant of the FCC lattice.
    - repetitions: The number of repetitions of the unit cell in each direction.
                [x, y, z]
    - vacuum: The thickness of the vacuum layer outside the plane perpendicular to the z-axis.

    Returns:
    - An Atoms object representing the FCC lattice.
    """

    # Define the positions of the atoms in the unit cell
    positions = [[0, 0, 0],
                 [0.5, 0.5, 0],
                 [0.5, 0, 0.5],
                 [0, 0.5, 0.5]]

    # Scale the positions by the lattice constant
    positions = [[x * lattice_constant for x in pos] for pos in positions]

    # Create the FCC lattice
    fcc_lattice = Atoms([symbol] * 4, positions=positions, cell=[lattice_constant] * 3, pbc=[True, True, False])

    # Repeat the unit cell in x and y directions
    fcc_lattice *= repetitions

    # Add vacuum layer outside the plane perpendicular to the z-axis
    # z_max = fcc_lattice.positions[:, 2].max()
    # vacuum_layer = Atoms('He', positions=[[0, 0, z_max + vacuum]])
    # vacuum_layer *= [repetitions[0], repetitions[1], 1]
    # vacuum_layer.set_cell([fcc_lattice.get_cell()[0], fcc_lattice.get_cell()[1], vacuum], scale_atoms=False)
    # fcc_lattice += vacuum_layer

    return fcc_lattice


In [14]:
# test
test_lattice = create_fcc_lattice('Cu', 3.6, [10, 10, 3], 3)
write('test_lattice.xyz', test_lattice)


/tmp/ipykernel_30705/2122771116.py:30: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  fcc_lattice *= repetitions


In [9]:
'''
The main goal of this function is to combine two xyz files into one file.
example :
1.xyz
  1 1 1
  2 2 2
  3 3 3
2.xyz
  3 3 3
  5 5 5
the result will be :
1.xyz
  1 1 1
  2 2 2
  3 3 3
  3 3 3
  5 5 5
'''
def combine_xyz_files(origin, add):
    with open(origin, 'r') as f1:
        lines1 = f1.readlines()
    with open(add, 'r') as f2:
        lines2 = f2.readlines()
    with open(origin, 'w') as f1:
        f1.writelines(lines1)
        f1.writelines(lines2)


In [87]:
# function to find the closest neighbors of a vacancy

def get_closest_neighbors(lattice, vacancy_index):
    """
    Get the indices of the closest neighboring atoms to a vacancy in a lattice.

    Args:
    - lattice: The lattice containing the vacancy and neighboring atoms.
    - vacancy_index: The index of the vacancy in the lattice.
    - num_neighbors: The number of neighboring atoms to return.

    Returns:
    - A list of the indices of the closest neighboring atoms to the vacancy.
    """
    # Create a neighbor list
    cutoff = neighborlist.natural_cutoffs(lattice)
    nl = neighborlist.NeighborList(cutoff, self_interaction=False, bothways=True)
    nl.update(lattice)

    # Get the indices of the atoms that are neighbors to the vacancy
    indices, offsets = nl.get_neighbors(vacancy_index)

    return indices




In [100]:
# function to diffusion of a vacancy
candidate_table = []    # list of candidate atoms. element: [start_pos, end_pos]
diffusion_table = []    # list of diffusion rate. Indices corresponding to the candidate_table  element: diffusion rate


def diffuse_one_step(lattice, vacancy_position):
    global candidate_table, diffusion_table
    # 1. find a candididate
    if vacancy_position:
        lattice.append(Atom('Cu', position=vacancy_position))
        candidate_idx = get_closest_neighbors(lattice, -1)
        del lattice[-1]
    # 2. 

In [55]:
# diffuse one step
test_lattice[0].position = [0, 1.8, -1.8]
write('test_lattice.xyz', test_lattice)


In [107]:
cand = diffuse_one_step(test_lattice, [0, 0, 0])

In [110]:
for idx in cand:
    print(test_lattice[idx].position, get_closest_neighbors(test_lattice, idx))

[ 0.   1.8 -1.8] [  12    1 1081]
[1.8 1.8 0. ] [  2  14  12   3 120 132 123   0]
[1.8 0.  1.8] [  3 120 123   4   5 124   1 109 111 113 231]
[0.  1.8 1.8] [  14   12    4    5   16    1    2 1081 1082 1085 1094]
[ 1.8 34.2  0. ] [111 110 228 231   2 120 108]
[ 0.  34.2  1.8] [ 112  113    2    4  108  109  110 1082 1189 1190 1193]
[34.2  1.8  0. ] [1083 1082 1092 1094   12    3    0 1080]
[34.2  0.   1.8] [1083 1084 1085  111    3    4 1080 1081 1189 1191 1193]
[34.2 34.2  0. ] [1190 1191 1082 1080  111  108 1188]


## Parameter setting
---

In [112]:
# parameter for diffusion rate
'''
bond energy : 200kJ/mol ~~ 2.07 eV/particle
fcc -> 12 nearest neighbors
'''
bond_energy = 2.07 
temperature = 500
e0 = 0.1
num_closest_neighbors = 12

# site energy, e_(bond number)
e_site = np.zeros(num_closest_neighbors)
for i in range(num_closest_neighbors):
    e_site[i] = get_site_energy(bond_energy, i)

# activation energy, e_a_(start to end)
e_a = np.zeros((num_closest_neighbors, num_closest_neighbors))
for i in range(num_closest_neighbors):
    for j in range(num_closest_neighbors):
        e_a[i, j] = get_activation_energy(e_site[i], e_site[j], e0=e0)

# diffusion rate, rate_(start to end)
diffusion_rate = np.zeros((num_closest_neighbors, num_closest_neighbors))
for i in range(num_closest_neighbors):
    for j in range(num_closest_neighbors):
        diffusion_rate[i, j] = get_diffusion_rate(e_a[i, j], temperature)
   

In [114]:
# value check
print(e_site)
print('-----------------')
# print(e_a)
print('-----------------')
print(diffusion_rate)

[  0.     -1.035  -2.07   -3.105  -4.14   -5.175  -6.21   -7.245  -8.28
  -9.315 -10.35  -11.385]
-----------------
-----------------
[[9.81848233e+011 1.08462210e+013 1.19815370e+014 1.32356909e+015
  1.46211220e+016 1.61515715e+017 1.78422191e+018 1.97098334e+019
  2.17729381e+020 2.40519960e+021 2.65696116e+022 2.93507559e+023]
 [3.28450128e+000 9.81848233e+011 1.08462210e+013 1.19815370e+014
  1.32356909e+015 1.46211220e+016 1.61515715e+017 1.78422191e+018
  1.97098334e+019 2.17729381e+020 2.40519960e+021 2.65696116e+022]
 [1.09873892e-011 3.28450128e+000 9.81848233e+011 1.08462210e+013
  1.19815370e+014 1.32356909e+015 1.46211220e+016 1.61515715e+017
  1.78422191e+018 1.97098334e+019 2.17729381e+020 2.40519960e+021]
 [3.67552667e-023 1.09873892e-011 3.28450128e+000 9.81848233e+011
  1.08462210e+013 1.19815370e+014 1.32356909e+015 1.46211220e+016
  1.61515715e+017 1.78422191e+018 1.97098334e+019 2.17729381e+020]
 [1.22954563e-034 3.67552667e-023 1.09873892e-011 3.28450128e+000
  9.